# Predict onset of diabetes using Pima Indians dataset

- Source 1: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/
- Source 2: https://machinelearningmastery.com/5-step-life-cycle-neural-network-models-keras/
- Dataset: https://archive.ics.uci.edu/ml/datasets/diabetes

# 1. Setup

In [1]:
# ! conda install keras -y

In [2]:
# ! conda install tensorflow -y

In [25]:
# imports
from numpy import loadtxt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf

In [4]:
# data source
# url = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv'
url = 'data/pima-indians-diabetes.data.csv'

In [5]:
# load the dataset
dataset = loadtxt(url, delimiter=',')

In [6]:
# without numpy
df = pd.read_csv(url, header=None)
df.columns = ['pregnancies', 'plasma glucose', 'blood pressure', 
              'skin fold', 'insulin', 'BMI', 'pedigree', 'age', 'onset']
df.shape

(768, 9)

In [7]:
# what the dataset looks like
df.head()

,pregnancies,plasma glucose,blood pressure,skin fold,insulin,BMI,pedigree,age,onset
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [8]:
# split into input (X) and output (y) variables
X = dataset[:,0:8]
y = dataset[:,8]

In [9]:
# examine the target
y[:5]

array([1., 0., 1., 0., 1.])

In [10]:
# split into 67% for train and 33% for test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

# 2. Define Keras Model

In [11]:
# keras has two APIs: Sequential and Functional.
model = Sequential()

2023-03-23 20:20:47.469507: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [12]:
# input layer
model.add(Dense(12, input_dim=8, activation='relu'))

In [13]:
# hidden layer
model.add(Dense(16, activation='relu'))

In [14]:
# output layer
model.add(Dense(1, activation='sigmoid'))

# 3. Compile Keras Model

In [15]:
# compile the keras model
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy']
             )

# 4. Fit the model

In [16]:
# fit the keras model on the dataset

model.fit(X_train, 
          y_train, 
          validation_data=(X_test,y_test), 
          epochs=10, 
          batch_size=10)

Epoch 1/10
52/52 [==============================] - 1s 6ms/step - loss: 7.8981 - accuracy: 0.6556 - val_loss: 4.0232 - val_accuracy: 0.6220
Epoch 2/10
52/52 [==============================] - 0s 2ms/step - loss: 2.1990 - accuracy: 0.6187 - val_loss: 1.6467 - val_accuracy: 0.6457
Epoch 3/10
52/52 [==============================] - 0s 2ms/step - loss: 1.2981 - accuracy: 0.6323 - val_loss: 1.3144 - val_accuracy: 0.6772
Epoch 4/10
52/52 [==============================] - 0s 2ms/step - loss: 1.1412 - accuracy: 0.6206 - val_loss: 1.1233 - val_accuracy: 0.6535
Epoch 5/10
52/52 [==============================] - 0s 2ms/step - loss: 0.9947 - accuracy: 0.6245 - val_loss: 0.9974 - val_accuracy: 0.6220
Epoch 6/10
52/52 [==============================] - 0s 2ms/step - loss: 0.8925 - accuracy: 0.6401 - val_loss: 0.8647 - val_accuracy: 0.6929
Epoch 7/10
52/52 [==============================] - 0s 2ms/step - loss: 0.7947 - accuracy: 0.6595 - val_loss: 0.7860 - val_accuracy: 0.7008
Epoch 8/10
52/52 [==

# 5. Make Predictions


In [17]:
# make probability predictions with the model
y_probs = model.predict(X_test)
# round predictions 
rounded = [round(x[0]) for x in y_probs]
rounded[:10]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [18]:
# sample 10 of those
import numpy as np
probs=y_probs.reshape(-1)
np.random.choice(probs, size=10)

array([0.7071251 , 0.9970852 , 0.26569235, 0.4750241 , 0.16412494,
       0.43127546, 0.33696085, 0.6512449 , 0.3096888 , 0.41192108],
      dtype=float32)

In [19]:
# make class predictions with the model
y_preds = (model.predict(X_test) > 0.5).astype(int)
print(y_preds[:10].tolist())
print(y_test[:10])

[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


# 6. Evaluate

#### Important note:
For a lot of reasons, the evaluation metrics will vary dramatically with each run of the model. A primary reason for this is the small size of the dataset. A smart solution is to use k-fold cross-validation, which will average out the results of multiple training runs (we do this in the next notebook).

In [20]:
# evaluate the keras model
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

8/8 [==============================] - 0s 911us/step - loss: 0.6681 - accuracy: 0.6850
Accuracy: 68.50


#### How to read the classification report
- Accuracy: The percentage of predictions that were accurate.
- Precision: Percentage of correct positive predictions relative to total positive predictions.
- Recall: Percentage of correct positive predictions relative to total actual positives.
- F1 Score: A weighted harmonic mean of precision and recall.
- Support: The number of occurrences of each class in y_test (i.e., how many observations belonged to each class in the test dataset).

In [21]:
# Evaluate the model
from sklearn import metrics
print(metrics.classification_report(y_test, y_preds))

              precision    recall  f1-score   support

         0.0       0.74      0.80      0.77       168
         1.0       0.54      0.45      0.49        86

    accuracy                           0.69       254
   macro avg       0.64      0.63      0.63       254
weighted avg       0.67      0.69      0.68       254



# 7. Save the model

In [22]:
model.save("diabetes-model-1.h5")

In [23]:
# load model
from keras.models import load_model
model2 = load_model("diabetes-model-1.h5")

# 8. Alternative: Categorical target as 2 variables

In [28]:
# split into input (X) and output (y) variables
from tensorflow.keras.utils import to_categorical
X = dataset[:,0:8]
y = to_categorical(dataset[:,8])

In [29]:
# examine the target & compare to the previous structure
print('before: \n', dataset[:,8][:5])
print('after: \n',  y[:5])

before: 
 [1. 0. 1. 0. 1.]
after: 
 [[0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]]


In [30]:
# split into 67% for train and 33% for test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [31]:
# build the model and fit to the data
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(16, activation='relu'))

In [32]:
# let's restructure the output layer
# previously we did this: 
#model.add(Dense(1, activation='sigmoid')) 
model.add(Dense(2, activation='softmax'))  
# notice the difference in nodes and activation function

In [33]:
# compile and fit
model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy']
             )
model.fit(X_train, 
          y_train, 
          validation_data=(X_test,y_test), 
          epochs=10, 
          batch_size=10)

Epoch 1/10
52/52 [==============================] - 1s 5ms/step - loss: 7.7312 - accuracy: 0.3930 - val_loss: 2.6831 - val_accuracy: 0.4882
Epoch 2/10
52/52 [==============================] - 0s 2ms/step - loss: 2.1120 - accuracy: 0.4514 - val_loss: 1.4216 - val_accuracy: 0.4685
Epoch 3/10
52/52 [==============================] - 0s 2ms/step - loss: 1.2059 - accuracy: 0.4669 - val_loss: 0.9384 - val_accuracy: 0.5512
Epoch 4/10
52/52 [==============================] - 0s 2ms/step - loss: 0.9774 - accuracy: 0.4981 - val_loss: 0.7856 - val_accuracy: 0.5906
Epoch 5/10
52/52 [==============================] - 0s 2ms/step - loss: 0.8241 - accuracy: 0.5720 - val_loss: 0.7083 - val_accuracy: 0.5984
Epoch 6/10
52/52 [==============================] - 0s 2ms/step - loss: 0.7355 - accuracy: 0.6051 - val_loss: 0.7016 - val_accuracy: 0.6732
Epoch 7/10
52/52 [==============================] - 0s 2ms/step - loss: 0.7386 - accuracy: 0.6031 - val_loss: 0.6486 - val_accuracy: 0.6693
Epoch 8/10
52/52 [==

In [34]:
# examine the predictions
y_probs = model.predict(X_test)
print(y_probs[:5])

y_preds = (model.predict(X_test) > 0.5).astype(int)
print(y_preds[:5])

[[0.41059807 0.58940196]
 [0.5652835  0.43471646]
 [0.48358864 0.51641136]
 [0.70919955 0.29080048]
 [0.7514305  0.24856944]]
[[0 1]
 [1 0]
 [0 1]
 [1 0]
 [1 0]]


In [35]:
# reformat the outcomes to a single value (this is optional)
y_test=[y[1] for y in y_test]
y_preds=[pred[1] for pred in y_preds]

In [36]:
# evaluate
print(metrics.classification_report(y_test, y_preds))

              precision    recall  f1-score   support

         0.0       0.78      0.76      0.77       168
         1.0       0.55      0.58      0.56        86

    accuracy                           0.70       254
   macro avg       0.66      0.67      0.67       254
weighted avg       0.70      0.70      0.70       254

